# Create a basic text search application

> Get started with the Python API to create and modify Vespa applications

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/create-text-app.ipynb)

This self-contained tutorial will create a basic text search application from scratch based on the MS MARCO dataset, similar to our [text search tutorials](https://docs.vespa.ai/documentation/tutorials/text-search.html). 

## Install pyvespa

In [ ]:
!pip install pyvespa

## Document

We first create a `Document` instance containing the `Field`s that we want to store in the app. In this case we will keep the application simple and include only the `id`, the `title` and the `body` of the MS MARCO documents.

In [1]:
from vespa.package import Document, Field

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)

## Schema

The complete `Schema` of our application will be named `msmarco` and contains the `Document` instance that we defined above, the default `FieldSet` indicates that queries will look for matches by searching both in the titles and bodies of the documents. The default `RankProfile` indicates that all the matched documents will be ranked by the `nativeRank` expression involving the title and the body of the matched documents.

In [2]:
from vespa.package import Schema, FieldSet, RankProfile

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)

## Application package

Once the `Schema` is defined, all we have to do is to create our msmarco `ApplicationPackage`:

In [3]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name = "msmarco", schema=[msmarco_schema])

At this point, `app_package` contains all the relevant information required to create our MS MARCO text search app. We now need to deploy it.